<a href="https://colab.research.google.com/github/vines2021/Deep-Learning-Sentimental-Analysis/blob/main/dl_sentimental_analysis_using_manual_method.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
data=pd.read_csv("/content/Book12.csv")
data

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1
...,...,...
1192,"When this was released, I thought this was one...",0
1193,I'd give this film a 1/10. This film is devoid...,0
1194,I was so surprised when I saw this film so muc...,1
1195,I was greatly disappointed by the quality of t...,0


In [ ]:
texts=data.review
labels=data.sentiment
print(texts)

0       One of the other reviewers has mentioned that ...
1       A wonderful little production. <br /><br />The...
2       I thought this was a wonderful way to spend ti...
3       Basically there's a family where a little boy ...
4       Petter Mattei's "Love in the Time of Money" is...
                              ...                        
1192    When this was released, I thought this was one...
1193    I'd give this film a 1/10. This film is devoid...
1194    I was so surprised when I saw this film so muc...
1195    I was greatly disappointed by the quality of t...
1196    I've tried to reconcile why so many bad review...
Name: review, Length: 1197, dtype: object


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

# Sample data (you should replace this with your own labeled dataset)
texts=data.review
labels=data.sentiment

# Tokenize text and create word index
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)
word_index = tokenizer.word_index
vocab_size = len(word_index) + 1

# Convert text to sequences and pad them to a fixed length
sequences = tokenizer.texts_to_sequences(texts)
max_sequence_length = max(len(seq) for seq in sequences)
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length)

# Convert labels to numpy array
labels = np.array(labels)

# Split the data into training and testing sets
split_ratio = 0.8
split_index = int(len(padded_sequences) * split_ratio)
x_train, x_test = padded_sequences[:split_index], padded_sequences[split_index:]
y_train, y_test = labels[:split_index], labels[split_index:]

# LSTM Model
embedding_dim = 100
lstm_units = 32

model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_sequence_length))
model.add(LSTM(units=lstm_units))
#--------
model.add(Dense(units=1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


epochs = 10
batch_size = 64

model.fit(x_train, y_train, epochs=50, batch_size=batch_size, validation_split=0.2)

# Evaluate the model
loss, accuracy = model.evaluate(x_test, y_test)
print(f"Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}")

# Make predictions
new_texts = [
    "I'm feeling better",
    "This product is awful.",
    "The performance was excellent.",
    "I'm really disappointed."
]

new_sequences = tokenizer.texts_to_sequences(new_texts)
new_padded_sequences = pad_sequences(new_sequences, maxlen=max_sequence_length)
predictions = model.predict(new_padded_sequences)

for i, text in enumerate(new_texts):
    sentiment = "Positive" if predictions[i] >= 0.5 else "Negative"
    print(f"'{text}' -> Predicted Sentiment: {sentiment}")

    print(predictions[i])


Epoch 1/50
12/12 [==============================] - 8s 410ms/step - loss: 0.6922 - accuracy: 0.5216 - val_loss: 0.6930 - val_accuracy: 0.4531
Epoch 2/50
12/12 [==============================] - 3s 273ms/step - loss: 0.6774 - accuracy: 0.7059 - val_loss: 0.6893 - val_accuracy: 0.5104
Epoch 3/50
12/12 [==============================] - 3s 259ms/step - loss: 0.6245 - accuracy: 0.7791 - val_loss: 0.6753 - val_accuracy: 0.5260
Epoch 4/50
12/12 [==============================] - 3s 292ms/step - loss: 0.4984 - accuracy: 0.9242 - val_loss: 0.6810 - val_accuracy: 0.5625
Epoch 5/50
12/12 [==============================] - 4s 344ms/step - loss: 0.3044 - accuracy: 0.9477 - val_loss: 0.5355 - val_accuracy: 0.7760
Epoch 6/50
12/12 [==============================] - 3s 216ms/step - loss: 0.1625 - accuracy: 0.9882 - val_loss: 0.5629 - val_accuracy: 0.7656
Epoch 7/50
12/12 [==============================] - 3s 271ms/step - loss: 0.0972 - accuracy: 0.9948 - val_loss: 0.7639 - val_accuracy: 0.7240
Epoch 

In [ ]:
print(accuracy)

0.7666666507720947
